In [16]:
# create spark session
import uuid
import time

from pyspark.sql import Row, DataFrame
from pyspark.sql.functions import explode, count,col, format_number
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, ArrayType

from app.common.spark_ext import *
from app.common.json_util import *
from app.common.s3 import *
from app.common.json_util import *

config = {
    "spark_conf_name": "spark_4",
    "skip_success_check": True,
    # "spark.dynamicAllocation.maxExecutors":120,
    "spark.executorEnv.HOME": "/share/shijin",
}

spark = new_spark_session("dingo-puyu3-hf", config)

/tmp/ipykernel_66237/2676430928.py:9: DeprecationWarning: `app.common.spark_ext` is deprecated, use `xinghe.spark` instead.
  from app.common.spark_ext import *
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/18 17:59:34 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


# 抽取数据

In [3]:
# read data
input_paths = [
    "s3://qa-huawei/shijin/15T/pjcc/sample_data/zh/"
]
input_df = read_any_path(spark, ",".join(input_paths), config)

In [5]:
input_df_total=input_df.count()
input_df_total

11748

In [17]:
# sample data
sample_df = input_df.sample(fraction=160/input_df_total).rdd.toDF().limit(150)
sample_df.count()

150

In [ ]:
output_acc = S3UploadAcc(spark.sparkContext)
output_path = "s3://qa-huawei/shijin/15T/pjcc/human_check/zh/"

sample_df.repartition(1).foreachPartition(upload_to_s3(output_path, "jsonl", output_acc, 0))

In [4]:
# process data
def process_data(row) -> Row:
    data = json_loads(row.value)
    content_qa = Doc(data).content
    data['content_qa'] = content_qa
    return Row(value=json_dumps(data))

process_df = sample_df.rdd.map(process_data).toDF()

In [5]:
# write data
output_path = "s3://qa-huawei/shijin/15T/pjcc/sample_data/en"
write_any_path(process_df, output_path, config)

{'rows': 12003,
 'bytes': {'sum': 595619325,
  'min': 3060,
  'max': 5073613,
  'cnt': 12003,
  'avg': 49622.538},
 'files': 1554,
 'sub_paths': {'en/en-pj-cc': {'rows': 12003,
   'bytes': {'sum': 595619325,
    'min': 3060,
    'max': 5073613,
    'cnt': 12003,
    'avg': 49622.538},
   'files': 1554}}}

# 机器质检

In [17]:
input_paths = [
    "s3://llm-users-phdd2/jiangwenhao/article/zh-web-tencent/v003/",
]

input_df = read_any_path(spark, ",".join(input_paths), config)


In [ ]:
#input_review_rdd = input_df.rdd.map(lambda x: {**json.loads(x.value), "filename": x.filename}).filter(lambda x : not x['content_qa']))

In [ ]:
# input_review_rdd.take(1)[0]

In [6]:
input_df.count()

157726394

In [18]:
from dingo.io import MetaData

input_rdd = input_df.rdd
input_rdd_format = input_rdd.map(lambda x: MetaData(
    data_id= str(json.loads(x.value)['track_id']),
    prompt='',
    content=json.loads(x.value)['content_qa'],
    # image=json.loads(x.value)['image'],
    # raw_data = json.loads(x.value)
))


In [19]:
from pyspark import SparkConf

from dingo.model.model import Model
from dingo.io import InputArgs
from dingo.exec import Executor

input_data = {
    "eval_group": "qa_standard_v1",
    'input_path': 'redpajama',
    'save_data': True,
    # 'save_raw': True
}
# input_data = {
#     "eval_group": "test",
#     "input_path": "redpajama",  # local filesystem dataset
#     "save_data": True,
#     "custom_config":
#         {
#             "rule_list": ["CommonSpecialCharacter", "CommonColonEnd"],
#             "prompt_list": ["LLMRepeat"],
#             "llm_config":
#                 {
#                     "detect_text_quality":
#                         {
#                             "key": "sk-proj-CwgXjGGBKq9vBI0Lue3jT3BlbkFJ5GnKDvQtrCmlwdHoea04",
#                             "api_url": "http://10.140.54.48:26666/v1",
#                         }
#                 }
#         }
# }
# Model.apply_config(input_data['custom_config_path'])
input_args = InputArgs(**input_data)
executor = Executor.exec_map["spark"](input_args, spark_session=spark, spark_rdd=input_rdd_format)
result = executor.execute()
result = result[0].to_dict()
print(result)

============= Init pyspark =============
============== Init Done ===============


[Stage 9:===================>                              (1390 + 1917) / 3623]

{'task_id': '5147a478-03e1-11f0-a9fe-e8ebd34ed3cc', 'task_name': 'dingo', 'eval_group': 'qa_standard_v1', 'input_path': '', 'output_path': '', 'create_time': '20250318_175946', 'finish_time': '20250318_181121', 'score': 100.0, 'num_good': 157725218, 'num_bad': 1176, 'total': 157726394, 'type_ratio': {'QUALITY_BAD_EFFECTIVENESS': 7e-06}, 'name_ratio': {'QUALITY_BAD_EFFECTIVENESS-RuleAbnormalChar': 3e-06, 'QUALITY_BAD_EFFECTIVENESS-RuleAbnormalHtml': 2e-06, 'QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl': 2e-06}}


In [133]:
import re
def is_url_only_content(content: str) -> bool:
    """判断内容是否全部由URL构成（改进分割逻辑）"""
    if not content.strip():
        return False
    
    # 分割策略：仅按空白符（空格、换行等）分割，保留URL内部字符
    segments = re.split(r"""\n+|[\s\'">]+|'""", content.strip())
    return all(
        is_single_url(segment)
        for segment in segments
        if segment.strip()  # 过滤空段落
    )

# 预编译正则表达式（网页1、网页2、网页3）

URL_PATTERN = re.compile(
    r'^'
    r'(?:(?:https?):\/\/)?'  # 协议头（非捕获组优化）
    r'(?:www\.)?'            # 可选www前缀（网页2非捕获组建议）
    r'(?:'                   # 主域名部分（原子组优化）
        r'(?:[a-zA-Z0-9-]{1,63}\.)+[a-zA-Z]{2,63}'  # 域名结构（网页1长度限制）
        r'|'                                        # 或IP地址
        r'(?:\d{1,3}\.){3}\d{1,3}'                 # IPv4（精确匹配）
        r'|'                                        # 或
        r'$$([a-f0-9:]+)$$'                        # IPv6（网页4特殊符号处理）
    r')'
    r'(?::\d{1,5})?'         # 端口号（限制1-5位数字）
    r'(?:\/'                 # 路径参数（网页3回溯预防）
        r'[\w$-_.+!*\'(),%;&=@~#]*'  # 安全字符集（排除回溯风险）
        r'(?:%[0-9a-fA-F]{2})*'      # 编码字符（替代宽泛匹配）
    r')*'                   # 允许多级路径
    r'$', 
    re.IGNORECASE | re.VERBOSE
)


def is_single_url(text: str) -> bool:
 
    """严格验证单个URL（支持IPv6/Punycode/特殊符号）"""
    cleaned = text.strip() # 清理边界符号（网页4的清理建议）
    url_pattern_2 = re.compile(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.IGNORECASE)
    tag = URL_PATTERN.fullmatch(cleaned) is not None
    tag2 = url_pattern_2.fullmatch(cleaned) is not None
     
    return tag2
 


In [134]:
 # or url_pattern_2.fullmatch(cleaned)

dict_1 = {'data_id': '56236c30-f7a7-4e68-89e6-04c3c6659d8c', 'prompt': '', 'content': 'http://xn--fiq8kv4t3f773c394b.xn--kput3i/\n', 'error_status': True, 'type_list': ['QUALITY_BAD_EFFECTIVENESS'], 'name_list': ['QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl'], 'reason_list': ['Content is only an url link.'], 'raw_data': {}}

In [135]:
is_url_only_content(dict_1['content'])

True

In [136]:
cnt =0
for i in url_rule:
    
    if is_url_only_content(i['content'])!=True :
        cnt+=1
        print(i)

print(cnt)

{'data_id': '730f4551-1578-448e-9b86-2e6f4956b51a', 'prompt': '', 'content': 'https://qjrboss.newaircloud.com/qjrb/video/202408/05/121198ec-ed0f-4799-9983-e00aa431941b.mp4?version=1722821725<!--PARAGRAPH_0--><!--PARAGRAPH_1--><!--PARAGRAPH_2--><!--PARAGRAPH_3--><!--PARAGRAPH_4-->\n', 'error_status': True, 'type_list': ['QUALITY_BAD_EFFECTIVENESS'], 'name_list': ['QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl'], 'reason_list': ['Content is only an url link.'], 'raw_data': {}}
{'data_id': '3bb6b07f-919c-4b12-b49e-7a05ed4e2dee', 'prompt': '', 'content': 'https://qjrboss.newaircloud.com/qjrb/video/202408/04/8293e665-b1eb-4563-bfc4-2cd26ecfd0e5.mp4?version=1722735460<!--PARAGRAPH_0--><!--PARAGRAPH_1--><!--PARAGRAPH_2--><!--PARAGRAPH_3--><!--PARAGRAPH_4-->\n', 'error_status': True, 'type_list': ['QUALITY_BAD_EFFECTIVENESS'], 'name_list': ['QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl'], 'reason_list': ['Content is only an url link.'], 'raw_data': {}}
2


In [23]:
# 4622

bad_info_list = executor.bad_info_list


In [8]:
spark

In [24]:
# QUALITY_BAD_EFFECTIVENESS-RuleInvisibleChar :1759  list code
# QUALITY_BAD_EFFECTIVENESS-RuleContentShort :  932 解决
# QUALITY_BAD_EFFECTIVENESS-RuleHtmlEntity :1722 table
# QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl : 181 已解决
# QUALITY_BAD_EFFECTIVENESS-RuleSpecialCharacter : 6 未发现
url_rule = bad_info_list.filter(lambda x :x['name_list'] ==['QUALITY_BAD_EFFECTIVENESS-RuleOnlyUrl']).take(500)

In [107]:
summary_path = "s3://llm-users/qa/hf/redpajama/summary_1121/"
write_any_path(result, summary_path, config)

Exception: s3 dir [s3://llm-users/qa/hf/redpajama/summary_1121/] is already existed.

In [6]:
bad_info_list = executor.bad_info_list

output_df = bad_info_list.map(lambda x: {"value":json_dumps(x)})
output_df = output_df.toDF()

# output_path = "s3://llm-users/qa/m10ap/error_info/"+ds
# write_any_path(output_df, output_path, config)

rule_name_list = Model.get_rules_by_group('qa_standard_v1')
for r in rule_name_list:
    # filter data
    def filter_data(row) -> bool:
        # true, keep
        return r in json_loads(row.value)["name_list"]
    try:
        filter_df = output_df.rdd.filter(filter_data).toDF()
    except ValueError:
        continue

    output_path = f"s3://llm-users/qa/hf/redpajama/error_info_1121/{r}"
    write_any_path(filter_df, output_path, config)

24/11/21 18:02:50 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_9_0 !
24/11/21 18:02:50 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_9_1 !


In [15]:
bad_info_list = executor.bad_info_list
list_rdd = bad_info_list.flatMap(lambda row: row['type_list'])
unique_list = list_rdd.distinct().collect()
unique_list

['QUALITY_INEFFECTIVENESS']

In [14]:
list_rdd = self.bad_info_list.flatMap(lambda row: row['type_list'])
unique_list = list_rdd.distinct().collect()

NameError: name 'self' is not defined